# Send Data From R to MongoDb with mongolite
`Mr Fugu Data Science`

____________________________

# (◕‿◕✿)

[Github](https://github.com/MrFuguDataScience) | [Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw?view_as=subscriber)

# Outcome & Purpose:

+ `Send Dataframes and .CSV() files to Mongo from R`
    + `Run Queries`
        + `Send data from Mongo to R`
  
[official mongolite manual](https://jeroen.github.io/mongolite/)

In [ ]:
# install.packages('mongolite')
# install.packages('rjson')

In [45]:
# 
library(knitr)     # help run code
library(markdown)  # create markdown files i.e. pdf
library(dplyr)
library(mongolite) # Create connection/Interface R<-> Mongodb
library(jsonlite)  # send files to Mongo

In [48]:
# This will not work for me, not sure why!

# mongo('mongodb:://localhost',db='fake_users',collection = 'user_info')

In [50]:
# Create Connection: 'localhost'
mng_conn<-mongo(collection = 'user_info',db='fake_users')

# Queries: `From Mongo to R`

In [52]:

# find all rows:
n<-mng_conn$find('{}')

# get a glimpse (idea)
dplyr::glimpse(n)


head(n)
# n

Rows: 5,826
Columns: 5
$ credit_card          <chr> "5399-3484-4724-7187", "1630-5261-6108-7631", "4…
$ email                <chr> "gso@qiegan.sqe", "xnji@gfruaxqnvm.fha", "dvyco@…
$ first_name           <chr> "Donyell Ann", "Bishop", "Connor", "Kylie", "Ant…
$ last_name            <chr> "Ospina", "Siyed", "Powers", "Her", "Vo", "Mares…
$ primary_phone_number <dbl> 5219459148, 4164254716, 3627413915, 3562764561, …


,credit_card,email,first_name,last_name,primary_phone_number
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
2,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
3,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
4,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561
5,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348
6,1459-9918-1722-7369,jofmezlbp@iw.evx,Mutammam,Mares,3247247289


In [54]:
#Count all rows
mng_conn$count()

# query anyone named "Alex"
specific_name<-mng_conn$find('{"first_name":"Bishop"}')
specific_name

[1] 5826

,credit_card,email,first_name,last_name,primary_phone_number
,<chr>,<chr>,<chr>,<chr>,<dbl>
1,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716


# Send Data `From Mongodb` to R:

In [55]:
# nv<-jsonlite::fromJSON('user_dump.json')

# # data.frame(nv)
# nv

# json_data <- fromJSON('user_dump.json', nullValue = NA)
# asFrame <- do.call("rbind.fill", lapply(json_data, as.data.frame))


require(RJSONIO)  

json_file <- fromJSON("user_dump.json")


# read_json("user_dump.json", simplifyVector = TRUE)

# Take a dumped json file from Mongo & `Import to R`

require(RJSONIO)  

json_file <- fromJSON("user_dump.json")

In [56]:

# str_replace_all(json_file,"\"_id[^,]*,\\s*" , "")

Lines <- readLines("user_dump.json") 
json_df <- as.data.frame(t(sapply(Lines, fromJSON)),row.names = F)
head(json_df)

,_id,credit_card,email,first_name,last_name,primary_phone_number
,<named list>,<named list>,<named list>,<named list>,<named list>,<named list>
1,5e94c2cba908db03dd29db4c,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
2,5e94c2cba908db03dd29db4d,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
3,5e94c2cba908db03dd29db4e,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
4,5e94c2cba908db03dd29db4f,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561
5,5e94c2cba908db03dd29db50,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348
6,5e94c2cba908db03dd29db51,1459-9918-1722-7369,jofmezlbp@iw.evx,Mutammam,Mares,3247247289


In [57]:
mng_conn$drop()
mng_conn$count()

[1] 0

# `Send Data.frame` From R to Mongodb

In [60]:
# read in csv()
user_df <- as.data.frame(read.csv('fake_users_R.csv'),header=TRUE,sep=',')[,-1]

head(user_df)

,credit_card,email,first_name,last_name,primary_phone_number
,<fct>,<fct>,<fct>,<fct>,<dbl>
1,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
2,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
3,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
4,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561
5,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348
6,1459-9918-1722-7369,jofmezlbp@iw.evx,Mutammam,Mares,3247247289


# Convert `Data.frame` to JSON and `Send to Mongo` from R:

In [61]:
x <- jsonlite::toJSON(user_df,collapse='',byrow=TRUE)
writeLines(x,sep='\n','ter.txt')
mng_conn$import(file('ter.txt','r'))

In [69]:
mng_conn$count()
# mng_conn$find('{}')

[1] 5826

In [64]:
# jsonlite::toJSON(user_df,byrow=TRUE,pretty = T)

# Send `.csv` to Mongo from R

# `Convert .CSV() to JSON` and `send to Mongo from R`:


In [70]:
write.csv(user_df,'wowzas.csv')

In [71]:
# write_json(read.csv("wowzas.csv",sep = ','), "R_dataframe.json",)

# toJSON('wowzas.csv')

wow_csv <- read.csv("wowzas.csv")
y<-jsonlite::toJSON(wow_csv,collapse='',byrow=TRUE)
writeLines(x,sep='\n','tert.txt')
mng_conn$import(file('tert.txt','r',))

In [72]:
mng_conn$count()

[1] 11652

In [ ]:
# This sends the whole 5826 Rows as 1 DOCUMENT: If you want to send individually:
# either iterate, or just do an insert!

# Import into Mongo from R:

In [73]:
# mng_conn$import(file('user_dump.txt','r'))
# mng_conn$count()
# mng_conn$find('{}')


# Lines <- readLines("try_json.json")
mng_conn$import(file('R_dataframe.json'),'r')


mng_conn$count()

[1] 11653

# `Insert Data` into MongoDB from R:
+ `Sending Data.frame()`

In [74]:

mng_conn$insert(user_df)
# mng_conn$count()
# mng_conn$drop()
mng_conn$count()

List of 5
 $ nInserted  : num 5826
 $ nMatched   : num 0
 $ nRemoved   : num 0
 $ nUpserted  : num 0
 $ writeErrors: list()

[1] 17479

In [ ]:
# write.csv(user_df,'wowzas.csv',row.names=FALSE)

# Citations:

# ಠ_ಠ

https://docs.mongodb.com/manual/core/gridfs/#faq-developers-when-to-use-gridfs

https://rstudio-pubs-static.s3.amazonaws.com/544823_1115c38053934765a7a2aa2131dab5f5.html

https://jeroen.github.io/mongolite/import-export.html#json

https://stackoverflow.com/questions/57136832/key-value-pairs-with-character-data-to-a-dataframe-in-r

https://www.travishinkelman.com/posts/reading-writing-json-files-r-racket/

https://jeroen.github.io/mongolite/manipulate-data.html#insert